In [2]:
import tensorflow as tf; print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))

2023-09-19 03:07:24.106469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 03:07:24.817039: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cuda_nvrtc/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cublas/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/:/usr/local/cudnn-112-81133/lib64:/u

Num GPUs Available:  4


In [1]:
import os

GPU = 3
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]= f"{GPU}"

import tensorflow as tf
tf.config.list_logical_devices('GPU')

2023-09-22 16:06:32.675980: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 16:06:33.445800: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cuda_nvrtc/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cublas/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/:/usr/local/cudnn-112-81133/lib64:/u

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [2]:
import sys
sys.path.append('./.local/lib/python3.8/site-packages')
sys.path.append('/usr/lib/python3/dist-packages')

from keras import losses, metrics
from keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Dropout, Activation, Input
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D, concatenate, Add
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import itertools as it
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats, interp
import os, sys, pickle, shutil
import pandas as pd
import numpy as np
import random, datetime, time
from sklearn.model_selection import KFold
#from keras.utils.training_utils import multi_gpu_model


# tensorflow 사용 시 seed 고정
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
SEED = 98

print(datetime.datetime.now())

2023-09-22 16:06:35.477667


# Input loading

## signals only

In [3]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'child'
NORM = 'z-norm'
GENDER = 'male'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')

x train shape: (7022, 5000, 12)
x test shape: (1756, 5000, 12)


In [4]:
SCALE_Y = 20
y_train = y_train / SCALE_Y
y_test = y_test / SCALE_Y

In [5]:
hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'

train_mask = (g_train == 1) if GENDER == 'male' else (g_train == 0)
test_mask = (g_test == 1) if GENDER == 'male' else (g_test == 0)

x_train = x_train[train_mask]
y_train = y_train[train_mask]
c_train = c_train[train_mask]
g_train = g_train[train_mask]

x_test = x_test[test_mask]
y_test = y_test[test_mask]
c_test = c_test[test_mask]
g_test = g_test[test_mask]

print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')

x train shape: (3891, 5000, 12)
x test shape: (963, 5000, 12)


In [ ]:
x_train = np.load(input_path+'x_train.npz', allow_pickle=True)['arr_0']
x_test = np.load(input_path+'x_test.npz', allow_pickle=True)['arr_0']
y_train = np.load(input_path+'y_train.npz')['arr_0']
y_test = np.load(input_path+'y_test.npz')['arr_0']
c_train = np.load(input_path+'c_train.npz')['arr_0']
c_test = np.load(input_path+'c_test.npz')['arr_0']

print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')

## signal+features

In [3]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'child'
NORM = 'z-norm'
GENDER = 'male'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/signal+features/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
     
with np.load(input_path+'feats.npz', allow_pickle=True) as f:
    feat_train = f['feat_train']
    feat_test = f['feat_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'

train_mask = (g_train == 1) if GENDER == 'male' else (g_train == 0)
test_mask = (g_test == 1) if GENDER == 'male' else (g_test == 0)

x_train = x_train[train_mask]
feat_train = feat_train[train_mask]
y_train = y_train[train_mask]
c_train = c_train[train_mask]
g_train = g_train[train_mask]

x_test = x_test[test_mask]
feat_test = feat_test[test_mask]
y_test = y_test[test_mask]
c_test = c_test[test_mask]
g_test = g_test[test_mask]

print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')


SCALE_Y = 100 if ADULT == 'adult' else 20
y_train = y_train / SCALE_Y
y_test = y_test / SCALE_Y

print(f'train: {np.mean(y_train)}, test: {np.mean(y_test)}')

x train shape: (7021, 5000, 12)
feat train shape: (7021, 55)
x test shape: (1756, 5000, 12)
feat test shape: (1756, 55)
x train shape: (3890, 5000, 12)
x test shape: (963, 5000, 12)
train: 0.10179626941680908, test: 0.10746798664331436


## signal+features+aug

In [6]:
import numpy as np
from tqdm import tqdm

class lead_extractor:
    """
    used to select specific leads or random choice of configurations

    Twelve leads: I, II, III, V1, V2, V3, V4, V5, V6, aVR, aVL, aVF, 
    Six leads: I, II, V1, V3, V4, V6
    Four leads: III, V1, V2, aVL # I, II, III, V2
    Three leads: I, II, V2
    Two leads: I, II

    """
    L2 = np.array([1,1,0,0,0,0,0,0,0,0,0,0])
    L3 = np.array([1,1,0,0,0,0,0,1,0,0,0,0])
    L4 = np.array([0,0,1,1,1,0,0,0,0,0,1,0]) #([1,1,1,0,0,0,0,1,0,0,0,0])
    L6 = np.array([1,1,0,1,0,1,1,0,1,0,0,0])
    L8 = np.array([1,1,0,1,0,1,1,1,1,0,1,0])
    L12 = np.array([1,1,1,1,1,1,1,1,1,1,1,1])

    @staticmethod
    def get (x,num_leads):
        if num_leads==None:
            # random choice output
            num_leads = random.choice([12,8,6,4,3,2])

        if num_leads==12:
            # Twelve leads: I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5, V6
            return x

        if num_leads==8:
            # Six leads: I, II, III, aVL, aVR, aVF
            x = x * lead_extractor.L8
            return x

        if num_leads==6:
            # Six leads: I, II, III, aVL, aVR, aVF
            x = x * lead_extractor.L6
            return x

        if num_leads==4:
            # Six leads: I, II, III, V2
            x = x * lead_extractor.L4
            return x

        if num_leads==3:
            # Three leads: I, II, V2
            x = x * lead_extractor.L3
            return x

        if num_leads==2:
            # Two leads: II, V5
            x = x * lead_extractor.L2
            return x
        raise Exception("invalid-leads-number")

def aug_generator(n_aug, x_train, feat_train, y_train, c_train, g_train):
    from tqdm import tqdm
    x_train_, feat_train_, y_train_, c_train_, g_train_ = [], [], [], [], []
    for i in tqdm(range(x_train.shape[0])):
        x_train_.append(x_train[i])
        feat_train_.append(feat_train[i])
        y_train_.append(y_train[i])
        c_train_.append(c_train[i])
        g_train_.append(g_train[i])

        x_train_.append(lead_extractor.get(x_train[i], 8))
        feat_train_.append(feat_train[i])
        y_train_.append(y_train[i])
        c_train_.append(c_train[i])
        g_train_.append(g_train[i])    

        if n_aug > 2:
            x_train_.append(lead_extractor.get(x_train[i], 6))
            feat_train_.append(feat_train[i])
            y_train_.append(y_train[i])
            c_train_.append(c_train[i])
            g_train_.append(g_train[i])  

        if n_aug > 3:
            x_train_.append(lead_extractor.get(x_train[i], 4))
            feat_train_.append(feat_train[i])
            y_train_.append(y_train[i])
            c_train_.append(c_train[i])
            g_train_.append(g_train[i]) 

    x_train_ = np.array(x_train_, np.float32)
    feat_train_ = np.array(feat_train_, np.float32)
    y_train_ = np.array(y_train_, np.float32)
    c_train_ = np.array(c_train_)
    g_train_ = np.array(g_train_, np.float32)

    print(f'x shape: {x_train_.shape}')
    print(f'feat shape: {feat_train_.shape}')
    print(f'y shape: {y_train_.shape}')
    
    return x_train_, feat_train_, y_train_, c_train_, g_train_

In [8]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'
GENDER = 'male'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/signal+features/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
     
with np.load(input_path+'feats.npz', allow_pickle=True) as f:
    feat_train = f['feat_train']
    feat_test = f['feat_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'

train_mask = (g_train == 1) if GENDER == 'male' else (g_train == 0)
test_mask = (g_test == 1) if GENDER == 'male' else (g_test == 0)

x_train = x_train[train_mask]
feat_train = feat_train[train_mask]
y_train = y_train[train_mask]
c_train = c_train[train_mask]
g_train = g_train[train_mask]

x_test = x_test[test_mask]
feat_test = feat_test[test_mask]
y_test = y_test[test_mask]
c_test = c_test[test_mask]
g_test = g_test[test_mask]

print(f'x train shape: {x_train.shape}')
print(f'y train shape: {y_train.shape}')
print(f'x test shape: {x_test.shape}')

x_train, feat_train, y_train, c_train, g_train = aug_generator(2, x_train, feat_train, y_train, c_train, g_train)
#x_test, feat_test, y_test, c_test, g_test = aug_generator(x_test, feat_test, y_test, c_test, g_test)

SCALE_Y = 100 if ADULT == 'adult' else 20
y_train = y_train / SCALE_Y
y_test = y_test / SCALE_Y

print(f'train: {np.mean(y_train)}, test: {np.mean(y_test)}')

x train shape: (27824, 5000, 12)
feat train shape: (27824, 55)
x test shape: (6962, 5000, 12)
feat test shape: (6962, 55)
x train shape: (13358, 5000, 12)
y train shape: (13358,)
x test shape: (3387, 5000, 12)


100%|██████████| 13358/13358 [00:03<00:00, 3800.82it/s]


x shape: (26716, 5000, 12)
feat shape: (26716, 55)
y shape: (26716,)
train: 0.6191248893737793, test: 0.6184705495834351


# Settings

In [6]:
# folder
nfold = 4  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 500
rootdir = f"randomSearch/{hyper_path}/CNN_4layers_largekernel(7~19)_age%10(sigmoid)_mae(nodecay)_{nfold}fold_test{ntest}"

if not os.path.exists(f"randomSearch/{hyper_path}"):
    os.mkdir(f"randomSearch/{hyper_path}")

if not os.path.exists(rootdir):
    os.mkdir(rootdir)

# 모델에 대한 정보 txt로 저장
f = open(f'{rootdir}/README.txt', 'w')
f.write(f'model: 1D CNN 4 layers, regression')
f.write(f'input: ECG of 10 second, output: age')
f.close()
    

# test_settings
layer_settings, test_settings = [], []


# hyperparamters
globalpool_opts = ['max','ave']

# hyperparamters pool
filt_opts = [16, 32, 64, 128] # num of filters(kernel)
stride_opts = [1,2,4]  # other opts: stride = (kernel-1)/2
kernel_opts = range(7,20,2) # kernel size
pool_size = 2
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [0, 16, 32, 64, 128]
BATCH_SIZE = [32, 64, 128, 256, 512]
lr_opts = [0.001, 0.002, 0.0005]

print('start making test settings...', end='', flush=True)
# test settings
nfilt, kernels, strides = [], [], []
for i in range(5):
    nfilt.append(0)
    kernels.append(0)
    strides.append(0)

for nfilter in filt_opts:
    for kernel in kernel_opts:
        for stride in stride_opts:
        #layer_settings.append([nfilter, kernel, int((kernel-1)/2)])       
            layer_settings.append([nfilter, kernel, stride])
    
for dense_node in dense_opts:
    for dropout_cnn in dropout_opts:
        for dropout_fc in dropout_opts:
            for batch_size in BATCH_SIZE:
                for learning_rate in lr_opts:
                    test_settings.append([dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate])                                   

                        
print('done')
print(datetime.datetime.now())

start making test settings...done
2023-09-21 11:07:17.735937


## 1d-cnn

In [ ]:
from keras import metrics
from tensorflow.keras.callbacks import LearningRateScheduler

seed_everything(SEED)

# random search for hyperparameter
ntrial = ntest
train_errs, val_errs = [] ,[]
test_acc, test_roc, test_prc = [], [], []
#test_rmse, test_mae, test_auc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    
    # total conv layers of the model
    n_conv = random.choice([2,3,4]) 
    # test settings
    for i in range(n_conv):
        nfilt[i], kernels[i], strides[i] = random.choice(layer_settings)
    dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate = random.choice(test_settings)
    
    if itrial < 0:
        continue
    

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = f'batch{batch_size},'
    for i in range(n_conv):
        odir_f += f'layer{i+1}:c{nfilt[i]}filt{kernels[i]}str{strides[i]},'
    odir_f += f'1conv,dropout{dropout_cnn},dnodes{dense_node},dropout{dropout_fc}'#,lr{learning_rate}'
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

        
    # model validation (VGG)
    out_shape = x_train.shape[1]
    for i in range(n_conv):
        out_shape = out_shape / pool_size / strides[i]
    if out_shape < 1:
        print('non-valid structure')
        os.rmdir(odir)
        itrial -= 1
        continue
        
    #mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/GPU:0","/GPU:1","/GPU:2","/GPU:3"])
    #with mirrored_strategy.scope(): 
    
    with tf.device(f'/gpu:{GPU}'):
        # build a model
        inp = Input(shape=(x_train.shape[1],x_train.shape[2]))
        out = inp


        # VGC block
        out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation=None)(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
        out = MaxPooling1D(pool_size)(out)


        # globalpooling vs flattening vs 1x1 convolution
        #elif globalpool_opt == 'ave':
        #    out = GlobalAveragePooling1D()(out)
        out = Conv1D(filters=1, kernel_size=1)(out)
        out = Flatten() (out)



        if dense_node != 0:
            out = Dropout(dropout_cnn)(out)
            out = Dense(dense_node, activation='relu')(out)
        out = Dropout(dropout_fc)(out)
        out = Dense(1, activation='sigmoid')(out)


        model = Model(inputs=[inp], outputs=[out])
        model.save_weights(f'{odir}/initial_weights.hdf5')


        # 4-fold cv
        kfold = KFold(nfold)
        tprs, aucs, prs = [], [], []
        test_rmse, test_mae = [], []
        f1_scores, thvals = [], []
        mean_fpr = np.linspace(0,1,100)
        mean_recall = np.linspace(0,1,100)

        switch = 0
        caseids_train = np.unique(c_train)
        for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
            c_cv_trains = caseids_train[c_cv_trains_mask]
            
            cv_train_mask = np.isin(c_train, c_cv_trains)
            cv_val_mask = ~cv_train_mask
            
            X_train = x_train[cv_train_mask]
            X_val = x_train[cv_val_mask]

            Y_train = y_train[cv_train_mask]
            Y_val = y_train[cv_val_mask]


            # model 학습
            try:
                # learning scheduler
                def step_decay(epoch):
                    start = 1e-3
                    drop = 0.1
                    epochs_drop = 10
                    lr = start * (drop ** np.floor((epoch)/epochs_drop))
                    return lr
                lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
                
                weightcache = f"{odir}/weights_{fold}.hdf5"
                
                model.compile(loss='mean_absolute_error', optimizer=Adam(lr=lr_scheduler, weight_decay=0), metrics=['mean_squared_error'])
                hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                        callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                    EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

                model.load_weights(weightcache)
                y_pred = model.predict(x_test).flatten()

                # MAE 계산
                model_err = metrics.MeanAbsoluteError()
                model_err.update_state(y_test, y_pred)
                mae_val = model_err.result().numpy()
                test_mae.append(mae_val)

                # RMSE 계산
                model_err = metrics.RootMeanSquaredError() 
                model_err.update_state(y_test, y_pred)
                rmse_val = model_err.result().numpy()
                test_rmse.append(rmse_val)


                print(f' ###{fold} fold : val mae {mae_val:.2f}, val rmse {rmse_val:.2f}###')
                tf.keras.backend.clear_session()
                model.load_weights(f'{odir}/initial_weights.hdf5')

            except Exception as e:
                print(e)
                switch = 1
                shutil.rmtree(odir)
                itrial -= 1
                break
        ###
        if switch:
            switch = 0
            tf.keras.backend.clear_session()
            continue

    # 

    # RMSE 계산
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)
    
    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))
    

    print(f'mae{mean_mae*10:.2f}+-{std_mae*10:.2f}_rmse{mean_rmse*10:.2f}+-{std_rmse*10:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*10:.2f}+-{std_mae*10:.2f}_rmse{mean_rmse*10:.2f}+-{std_rmse*10:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


random search 0/500


2023-09-21 11:09:19.136882: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 11:09:19.621664: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22324 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:c1:00.0, compute capability: 8.6


Epoch 1/100


2023-09-21 11:09:22.853037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-21 11:09:23.313032: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-21 11:09:23.725512: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-21 11:09:23.727835: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fccac0515f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-21 11:09:23.727882: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-21 11:09:23.736192: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-21 11:09:23.807048: I te

90/92 [============================>.] - ETA: 0s - loss: 0.1618 - mean_squared_error: 0.0618
Epoch 1: val_loss improved from inf to 0.16156, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN_4layers_largekernel(7~19)_age%10(sigmoid)_mae(nodecay)_4fold_test500/batch32,layer1:c128filt13str2,layer2:c16filt9str4,layer3:c64filt9str1,1conv,dropout0.1,dnodes64,dropout0/weights_0.hdf5
92/92 [==============================] - 6s 16ms/step - loss: 0.1618 - mean_squared_error: 0.0618 - val_loss: 0.1616 - val_mean_squared_error: 0.0622
Epoch 2/100
89/92 [============================>.] - ETA: 0s - loss: 0.1612 - mean_squared_error: 0.0616
Epoch 2: val_loss did not improve from 0.16156
92/92 [==============================] - 1s 12ms/step - loss: 0.1606 - mean_squared_error: 0.0613 - val_loss: 0.1616 - val_mean_squared_error: 0.0622
Epoch 3/100
89/92 [============================>.] - ETA: 0s - loss: 0.1605 - mean_squared_error: 0.0612
Epoch 3: val_loss did not improve 

...

Epoch 1/100
6/6 [==============================] - ETA: 0s - loss: 0.2462 - mean_squared_error: 0.1008
Epoch 1: val_loss improved from inf to 0.16682, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN_4layers_largekernel(7~19)_age%10(sigmoid)_mae(nodecay)_4fold_test500/batch512,layer1:c64filt17str2,layer2:c32filt11str2,1conv,dropout0.4,dnodes64,dropout0/weights_3.hdf5
6/6 [==============================] - 2s 147ms/step - loss: 0.2462 - mean_squared_error: 0.1008 - val_loss: 0.1668 - val_mean_squared_error: 0.0514
Epoch 2/100
5/6 [========================>.....] - ETA: 0s - loss: 0.1558 - mean_squared_error: 0.0567
Epoch 2: val_loss did not improve from 0.16682
6/6 [==============================] - 1s 117ms/step - loss: 0.1548 - mean_squared_error: 0.0560 - val_loss: 0.1804 - val_mean_squared_error: 0.0674
Epoch 3/100
5/6 [========================>.....] - ETA: 0s - loss: 0.1452 - mean_squared_error: 0.0514
Epoch 3: val_loss did not improve from 0.16682
6/

Epoch 1/100
92/92 [==============================] - ETA: 0s - loss: 0.1649 - mean_squared_error: 0.0650
Epoch 1: val_loss improved from inf to 0.16155, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN_4layers_largekernel(7~19)_age%10(sigmoid)_mae(nodecay)_4fold_test500/batch32,layer1:c64filt19str2,layer2:c64filt13str2,layer3:c128filt19str2,1conv,dropout0.1,dnodes64,dropout0.2/weights_0.hdf5
92/92 [==============================] - 3s 22ms/step - loss: 0.1649 - mean_squared_error: 0.0650 - val_loss: 0.1615 - val_mean_squared_error: 0.0622
Epoch 2/100
92/92 [==============================] - ETA: 0s - loss: 0.1605 - mean_squared_error: 0.0613
Epoch 2: val_loss did not improve from 0.16155
92/92 [==============================] - 2s 17ms/step - loss: 0.1605 - mean_squared_error: 0.0613 - val_loss: 0.1616 - val_mean_squared_error: 0.0622
Epoch 3/100
91/92 [============================>.] - ETA: 0s - loss: 0.1606 - mean_squared_error: 0.0614
Epoch 3: val_loss 

Epoch 1/100
89/92 [============================>.] - ETA: 0s - loss: 0.1631 - mean_squared_error: 0.0641
Epoch 1: val_loss improved from inf to 0.16526, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN_4layers_largekernel(7~19)_age%10(sigmoid)_mae(nodecay)_4fold_test500/batch32,layer1:c64filt19str2,layer2:c64filt13str2,layer3:c128filt19str2,1conv,dropout0.1,dnodes64,dropout0.2/weights_1.hdf5
92/92 [==============================] - 4s 21ms/step - loss: 0.1634 - mean_squared_error: 0.0641 - val_loss: 0.1653 - val_mean_squared_error: 0.0637
Epoch 2/100
89/92 [============================>.] - ETA: 0s - loss: 0.1595 - mean_squared_error: 0.0609
Epoch 2: val_loss did not improve from 0.16526
92/92 [==============================] - 1s 16ms/step - loss: 0.1594 - mean_squared_error: 0.0608 - val_loss: 0.1653 - val_mean_squared_error: 0.0637
Epoch 3/100
87/92 [===========================>..] - ETA: 0s - loss: 0.1597 - mean_squared_error: 0.0609
Epoch 3: val_loss 

Epoch 1/100
89/92 [============================>.] - ETA: 0s - loss: 0.1664 - mean_squared_error: 0.0652
Epoch 1: val_loss improved from inf to 0.15396, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN_4layers_largekernel(7~19)_age%10(sigmoid)_mae(nodecay)_4fold_test500/batch32,layer1:c64filt19str2,layer2:c64filt13str2,layer3:c128filt19str2,1conv,dropout0.1,dnodes64,dropout0.2/weights_2.hdf5
92/92 [==============================] - 3s 22ms/step - loss: 0.1663 - mean_squared_error: 0.0650 - val_loss: 0.1540 - val_mean_squared_error: 0.0587
Epoch 2/100
91/92 [============================>.] - ETA: 0s - loss: 0.1631 - mean_squared_error: 0.0625
Epoch 2: val_loss improved from 0.15396 to 0.15396, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN_4layers_largekernel(7~19)_age%10(sigmoid)_mae(nodecay)_4fold_test500/batch32,layer1:c64filt19str2,layer2:c64filt13str2,layer3:c128filt19str2,1conv,dropout0.1,dnodes64,dropout0.2/weights_2.h

Epoch 1/100
91/92 [============================>.] - ETA: 0s - loss: 0.1635 - mean_squared_error: 0.0639
Epoch 1: val_loss improved from inf to 0.16255, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN_4layers_largekernel(7~19)_age%10(sigmoid)_mae(nodecay)_4fold_test500/batch32,layer1:c64filt19str2,layer2:c64filt13str2,layer3:c128filt19str2,1conv,dropout0.1,dnodes64,dropout0.2/weights_3.hdf5
92/92 [==============================] - 3s 22ms/step - loss: 0.1634 - mean_squared_error: 0.0639 - val_loss: 0.1625 - val_mean_squared_error: 0.0616
Epoch 2/100
87/92 [===========================>..] - ETA: 0s - loss: 0.1601 - mean_squared_error: 0.0615
Epoch 2: val_loss did not improve from 0.16255
92/92 [==============================] - 2s 17ms/step - loss: 0.1603 - mean_squared_error: 0.0615 - val_loss: 0.1626 - val_mean_squared_error: 0.0616
Epoch 3/100
88/92 [===========================>..] - ETA: 0s - loss: 0.1605 - mean_squared_error: 0.0616
Epoch 3: val_loss 

## 1d-cnn + concat

In [ ]:
def signal_data_generator(data, batch_size):
    num_samples = len(data[0])
    num_batches = num_samples // batch_size

    while True:
        for idx in range(num_batches):
            batch_data = np.array(data)[:,idx * batch_size:(idx+1) * batch_size,:]

            # Perform preprocessing on batch_data if needed

            yield batch_data, labels  # Yield a batch of data and labels

In [9]:
# folder
nfold = 4  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 500
rootdir = f"randomSearch/{hyper_path}/CNN_4layers+feats_2aug_kernel(7~19)_age%{SCALE_Y}(sigmoid)_mae(decay)_{nfold}fold_test{ntest}"

if not os.path.exists(f"randomSearch/{hyper_path}"):
    os.mkdir(f"randomSearch/{hyper_path}")

if not os.path.exists(rootdir):
    os.mkdir(rootdir)

# 모델에 대한 정보 txt로 저장
f = open(f'{rootdir}/README.txt', 'w')
f.write(f'model: 1D CNN 4 layers, regression')
f.write(f'input: ECG of 10 second, output: age')
f.close()
    

# test_settings
layer_settings, test_settings = [], []


# hyperparamters
globalpool_opts = ['max','ave']

# hyperparamters pool
filt_opts = [16, 32, 64, 128] # num of filters(kernel)
stride_opts = [1,2,4]  # other opts: stride = (kernel-1)/2
kernel_opts = range(7,20,2) # kernel size
pool_size = 2
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [0, 16, 32, 64, 128]
BATCH_SIZE = [32, 64, 128, 256, 512]
lr_opts = [0.001, 0.002, 0.0005]

print('start making test settings...', end='', flush=True)
# test settings
nfilt, kernels, strides = [], [], []
for i in range(5):
    nfilt.append(0)
    kernels.append(0)
    strides.append(0)

for nfilter in filt_opts:
    for kernel in kernel_opts:
        for stride in stride_opts:
        #layer_settings.append([nfilter, kernel, int((kernel-1)/2)])       
            layer_settings.append([nfilter, kernel, stride])
    
for dense_node in dense_opts:
    for dropout_cnn in dropout_opts:
        for dropout_fc in dropout_opts:
            for batch_size in BATCH_SIZE:
                for learning_rate in lr_opts:
                    test_settings.append([dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate])                                   

                        
print('done')
print(datetime.datetime.now())

start making test settings...done
2023-09-22 16:10:20.964094


In [ ]:
from keras import metrics
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.layers import Concatenate
from sklearn.metrics import mean_absolute_error, mean_squared_error

seed_everything(SEED)

def step_decay(epoch):
    start = 1e-3
    drop = 0.1
    epochs_drop = 10
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr

# random search for hyperparameter
ntrial = ntest
train_errs, val_errs = [] ,[]
test_acc, test_roc, test_prc = [], [], []
#test_rmse, test_mae, test_auc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    
    # total conv layers of the model
    n_conv = random.choice([2,3,4]) 
    # test settings
    for i in range(n_conv):
        nfilt[i], kernels[i], strides[i] = random.choice(layer_settings)
    dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate = random.choice(test_settings)
    
    if itrial < 0:
        continue
    

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = f'batch{batch_size},'
    for i in range(n_conv):
        odir_f += f'layer{i+1}:c{nfilt[i]}filt{kernels[i]}str{strides[i]},'
    odir_f += f'1conv,dropout{dropout_cnn},dnodes{dense_node},dropout{dropout_fc}'#,lr{learning_rate}'
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

        
    # model validation (VGG)
    out_shape = x_train.shape[1]
    for i in range(n_conv):
        out_shape = out_shape / pool_size / strides[i]
    if out_shape < 1:
        print('non-valid structure')
        os.rmdir(odir)
        itrial -= 1
        continue
        
    #mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/GPU:0","/GPU:1","/GPU:2","/GPU:3"])
    #with mirrored_strategy.scope(): 
    
    with tf.device(f'/gpu:{GPU}'):
        # build a model
        inp = Input(shape=(x_train.shape[1],x_train.shape[2]), name="input_1")
        out = inp


        # VGC block
        for i in range(n_conv):
            out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = Activation('relu')(out)
            out = MaxPooling1D(pool_size)(out)


        # globalpooling vs flattening vs 1x1 convolution
        #elif globalpool_opt == 'ave':
        #    out = GlobalAveragePooling1D()(out)
        out = Conv1D(filters=1, kernel_size=1)(out)
        out = Flatten() (out)

    
        # concat with X_feature
        inp_feat = Input(shape=(feat_train.shape[1]), name="input_2")
        out = Concatenate(axis=1)([out, inp_feat])

        if dense_node != 0:
            out = Dropout(dropout_cnn)(out)
            out = Dense(dense_node, activation='relu')(out)
        out = Dropout(dropout_fc)(out)
        out = Dense(1, activation='sigmoid')(out)


        model = Model(inputs=[inp, inp_feat], outputs=[out])
        lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
        model.compile(loss='mean_absolute_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=[])
        model.save_weights(f'{odir}/initial_weights.hdf5')


    # 4-fold cv
    kfold = KFold(nfold)
    tprs, aucs, prs = [], [], []
    test_rmse, test_mae = [], []
    f1_scores, thvals = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)

    switch = 0
    caseids_train = np.unique(c_train)
    for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
        c_cv_trains = caseids_train[c_cv_trains_mask]

        cv_train_mask = np.isin(c_train, c_cv_trains)
        cv_val_mask = ~cv_train_mask

        X_train = x_train[cv_train_mask]
        X_val = x_train[cv_val_mask]

        Y_train = y_train[cv_train_mask]
        Y_val = y_train[cv_val_mask]

        Feat_train = feat_train[cv_train_mask]
        Feat_val = feat_train[cv_val_mask]

        
        # model 학습
        try:

            with tf.device("/CPU:0"):
                train_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_train, "input_2": Feat_train}, Y_train)).shuffle(x_train.shape[0]).batch(batch_size)
                val_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_val, "input_2": Feat_val}, Y_val)).batch(batch_size)
                test_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": x_test, "input_2": feat_test}, y_test)).batch(batch_size)
       
            weightcache = f"{odir}/weights_{fold}.hdf5"
            #model = multi_gpu_model(model, gpus=4)
            #model.compile(loss='mean_squared_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_absolute_error'])
            hist = model.fit(train_dataset, validation_data=val_dataset, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(test_dataset).flatten()

            # MAE 계산
            mae_val = mean_absolute_error(y_test, y_pred)
            test_mae.append(mae_val)


            print(f' ###{fold} fold : val mae {mae_val:.2f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

        except Exception as e:
            print(e)
            switch = 1
            shutil.rmtree(odir)
            itrial -= 1

            break
    ###
    if switch:
        switch = 0
        tf.keras.backend.clear_session()
        continue

    # 

    
    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))
    

    print(f'mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


random search 0/500


Epoch 1/100


2023-09-22 16:10:47.985906: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-22 16:10:48.503966: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-22 16:10:48.599388: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-22 16:10:48.601039: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7efd34567110 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-22 16:10:48.601055: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-22 16:10:48.605436: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-22 16:10:48.680387: I te

180/627 [=======>......................] - ETA: 6s - loss: 0.1142

In [14]:
strategy = tf.distribute.MirroredStrategy()
with stratetgy.scope

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2

In [20]:
tf.config.list_logical_devices('GPU')

[LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:GPU:1', device_type='GPU')]

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5000, 12)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 2500, 128)    20096       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 2500, 128)   512         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 2500, 128)    0           ['batch_normalization[0][0]']

## cnn transformer

In [8]:
random.seed(98)

from keras.models import Model
from keras.layers import Layer, LayerNormalization, Dense, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Input, concatenate, multiply, dot, MultiHeadAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler


SRATE = 500  # in hz
SEGLEN = 10 * SRATE  # samples
#BATCH_SIZE = 256
MAX_CASES = 500
nfold = 4

hyperparameters = {
    "nfilt" : [16, 32, 64],
    'nhead' : [2, 4, 8],
    'kdim': [16, 64, 128, 256],
    "fnode" : [32, 64, 128],
    "clayer" : [2, 3, 4],
    "tlayer" : [3, 4, 5],
    "droprate" : [0.1, 0.2, 0.5],
    "filtsize" : [5, 7, 9, 11, 15, 19],
    'poolsize' : [2, 4, 5],
    'batch_size': [64, 128, 256]
}
keys, values = zip(*hyperparameters.items())
permutations_dicts = it.product(*values)
permutations_dicts = list(permutations_dicts)
random.shuffle(permutations_dicts)
for nfilt, nhead, kdim, fnode, clayer, tlayer, droprate, filtsize, poolsize, batch_size in permutations_dicts:
    test_start = time.time()

    tf.keras.backend.clear_session()
    
    rootdir = f'randomSearch/{hyper_path}/CNN+transformer_age%{SCALE_Y}(sigmoid)_loss(mae)-nodecay_{nfold}fold_{MAX_CASES}cases'
    odir_f = 'batch{}_cnn{}_filt{}_size{}_pool{}_do{}'.format(batch_size, clayer, nfilt, filtsize, poolsize, droprate)
    odir_f += '_tra{}_head{}_kdim{}_fnn{}'.format(tlayer, nhead, kdim, fnode)
    
    if not os.path.exists(rootdir):
        os.mkdir(rootdir)
    
    odir = rootdir+'/'+odir_f
    print("============================")
    print(odir)
    print("============================")

    # cnn-transformer
    # https://keras.io/examples/nlp/text_classification_with_transformer/
    out = inp = Input(shape=(x_train.shape[1], x_train.shape[2]))
    
    # out = Conv1D(filters=kdim, kernel_size=filtsize, padding='same')(out)

    # conv 여러층    
    for i in range(clayer):
        out = Conv1D(filters=nfilt, kernel_size=filtsize, padding='same', activation=None)(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
        out = MaxPooling1D(poolsize, padding='same')(out)
    out = Dense(kdim)(out)  # 마지막 차원이 nfilt 인데 kdim 으로 바꿔야 transformer block을 쌓을 수 있다.
    for i in range(tlayer):  # transformer
        attn_output = MultiHeadAttention(num_heads=nhead, key_dim=kdim, attention_axes=[1,])(out, out)
        attn_output = Dropout(droprate)(attn_output)
        out1 = LayerNormalization(epsilon=1e-6)(out + attn_output)  # sum and norm
        ffn_output = tf.keras.Sequential([Dense(fnode, activation="relu"), Dense(kdim)])(out1)
        out2 = Dropout(droprate)(ffn_output)
        out = LayerNormalization(epsilon=1e-6)(out1 + out2)  # sum and norm
    out = GlobalMaxPooling1D()(out)

    if droprate:
        out = Dropout(droprate)(out)
    out = Dense(fnode)(out)
    if droprate:
        out = Dropout(droprate)(out)
    out = Dense(1, activation='sigmoid')(out)

    if not os.path.exists(odir):
        os.mkdir(odir)


    model = Model(inputs=[inp], outputs=[out])
    model.save_weights(f'{odir}/initial_weights.hdf5')
    
    
    # 4-fold cv
    kfold = KFold(nfold)
    tprs, aucs, prs = [], [], []
    test_rmse, test_mae = [], []
    f1_scores, thvals = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)

    switch = 0
    caseids_train = np.unique(c_train)
    for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
        c_cv_trains = caseids_train[c_cv_trains_mask]

        cv_train_mask = np.isin(c_train, c_cv_trains)
        cv_val_mask = ~cv_train_mask

        X_train = x_train[cv_train_mask]
        X_val = x_train[cv_val_mask]

        Y_train = y_train[cv_train_mask]
        Y_val = y_train[cv_val_mask]


        # model 학습
        try:
            # learning scheduler
            def step_decay(epoch):
                start = 1e-3
                drop = 0.1
                epochs_drop = 10
                lr = start * (drop ** np.floor((epoch)/epochs_drop))
                return lr
            lr_scheduler = LearningRateScheduler(step_decay, verbose=1)

            weightcache = f"{odir}/weights_{fold}.hdf5"
            #model = multi_gpu_model(model, gpus=4)
            model.compile(loss='mae', optimizer=Adam(lr=lr_scheduler, weight_decay=None), metrics=['mean_squared_error'])
            hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(x_test).flatten()

            # MAE 계산
            model_err = metrics.MeanAbsoluteError()
            model_err.update_state(y_test, y_pred)
            mae_val = model_err.result().numpy()
            test_mae.append(mae_val)


            print(f' ###{fold} fold : val mae {mae_val:.2f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

        except Exception as e:
            print(e)
            switch = 1
            shutil.rmtree(odir)
            break
    ###
    if switch:
        switch = 0
        tf.keras.backend.clear_session()
        continue


    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))


    print(f'mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128


Epoch 1/100
23/23 [==============================] - ETA: 0s - loss: 0.1068 - mean_squared_error: 0.0273
Epoch 1: val_loss improved from inf to 0.07329, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_0.hdf5
23/23 [==============================] - 19s 355ms/step - loss: 0.1068 - mean_squared_error: 0.0273 - val_loss: 0.0733 - val_mean_squared_error: 0.0097
Epoch 2/100
23/23 [==============================] - ETA: 0s - loss: 0.0787 - mean_squared_error: 0.0120
Epoch 2: val_loss improved from 0.07329 to 0.07197, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_0.hdf5
23/23 [==============================] - 7s 308ms/step - loss: 0.0787 - mean_squared_error: 0.0120 -

Epoch 1/100
23/23 [==============================] - ETA: 0s - loss: 0.1066 - mean_squared_error: 0.0272
Epoch 1: val_loss improved from inf to 0.07314, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_1.hdf5
23/23 [==============================] - 19s 354ms/step - loss: 0.1066 - mean_squared_error: 0.0272 - val_loss: 0.0731 - val_mean_squared_error: 0.0106
Epoch 2/100
23/23 [==============================] - ETA: 0s - loss: 0.0788 - mean_squared_error: 0.0121
Epoch 2: val_loss did not improve from 0.07314
23/23 [==============================] - 7s 322ms/step - loss: 0.0788 - mean_squared_error: 0.0121 - val_loss: 0.0783 - val_mean_squared_error: 0.0093
Epoch 3/100
23/23 [==============================] - ETA: 0s - loss: 0.0781 - mean_squared_error: 0.0121
Epoch 3: val_loss did not improve from 0.07314
23/23 [==============

Epoch 1/100
23/23 [==============================] - ETA: 0s - loss: 0.1052 - mean_squared_error: 0.0264
Epoch 1: val_loss improved from inf to 0.07653, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_2.hdf5
23/23 [==============================] - 19s 344ms/step - loss: 0.1052 - mean_squared_error: 0.0264 - val_loss: 0.0765 - val_mean_squared_error: 0.0088
Epoch 2/100
23/23 [==============================] - ETA: 0s - loss: 0.0798 - mean_squared_error: 0.0125
Epoch 2: val_loss improved from 0.07653 to 0.07459, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_2.hdf5
23/23 [==============================] - 7s 310ms/step - loss: 0.0798 - mean_squared_error: 0.0125 -

Epoch 1/100
23/23 [==============================] - ETA: 0s - loss: 0.1070 - mean_squared_error: 0.0275
Epoch 1: val_loss improved from inf to 0.07253, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_3.hdf5
23/23 [==============================] - 19s 355ms/step - loss: 0.1070 - mean_squared_error: 0.0275 - val_loss: 0.0725 - val_mean_squared_error: 0.0093
Epoch 2/100
23/23 [==============================] - ETA: 0s - loss: 0.0802 - mean_squared_error: 0.0123
Epoch 2: val_loss did not improve from 0.07253
23/23 [==============================] - 7s 304ms/step - loss: 0.0802 - mean_squared_error: 0.0123 - val_loss: 0.0755 - val_mean_squared_error: 0.0089
Epoch 3/100
23/23 [==============================] - ETA: 0s - loss: 0.0783 - mean_squared_error: 0.0122
Epoch 3: val_loss did not improve from 0.07253
23/23 [==============

...

Epoch 1/100
45/46 [============================>.] - ETA: 0s - loss: 0.1088 - mean_squared_error: 0.0242
Epoch 1: val_loss improved from inf to 0.10280, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch64_cnn3_filt32_size15_pool5_do0.1_tra5_head8_kdim256_fnn64/weights_0.hdf5
46/46 [==============================] - 17s 76ms/step - loss: 0.1087 - mean_squared_error: 0.0242 - val_loss: 0.1028 - val_mean_squared_error: 0.0196
Epoch 2/100
45/46 [============================>.] - ETA: 0s - loss: 0.1014 - mean_squared_error: 0.0193
Epoch 2: val_loss did not improve from 0.10280
46/46 [==============================] - 2s 47ms/step - loss: 0.1015 - mean_squared_error: 0.0193 - val_loss: 0.1028 - val_mean_squared_error: 0.0196
Epoch 3/100
45/46 [============================>.] - ETA: 0s - loss: 0.1013 - mean_squared_error: 0.0193
Epoch 3: val_loss did not improve from 0.10280
46/46 [================

Epoch 1/100
45/46 [============================>.] - ETA: 0s - loss: 0.1076 - mean_squared_error: 0.0233
Epoch 1: val_loss improved from inf to 0.10339, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch64_cnn3_filt32_size15_pool5_do0.1_tra5_head8_kdim256_fnn64/weights_1.hdf5
46/46 [==============================] - 16s 68ms/step - loss: 0.1073 - mean_squared_error: 0.0231 - val_loss: 0.1034 - val_mean_squared_error: 0.0200
Epoch 2/100
45/46 [============================>.] - ETA: 0s - loss: 0.1012 - mean_squared_error: 0.0192
Epoch 2: val_loss did not improve from 0.10339
46/46 [==============================] - 2s 47ms/step - loss: 0.1013 - mean_squared_error: 0.0192 - val_loss: 0.1034 - val_mean_squared_error: 0.0200
Epoch 3/100
46/46 [==============================] - ETA: 0s - loss: 0.1013 - mean_squared_error: 0.0192
Epoch 3: val_loss did not improve from 0.10339
46/46 [================

Epoch 1/100
45/46 [============================>.] - ETA: 0s - loss: 0.1105 - mean_squared_error: 0.0250
Epoch 1: val_loss improved from inf to 0.09845, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch64_cnn3_filt32_size15_pool5_do0.1_tra5_head8_kdim256_fnn64/weights_2.hdf5
46/46 [==============================] - 20s 86ms/step - loss: 0.1104 - mean_squared_error: 0.0249 - val_loss: 0.0984 - val_mean_squared_error: 0.0185
Epoch 2/100
45/46 [============================>.] - ETA: 0s - loss: 0.1032 - mean_squared_error: 0.0198
Epoch 2: val_loss did not improve from 0.09845
46/46 [==============================] - 3s 59ms/step - loss: 0.1029 - mean_squared_error: 0.0197 - val_loss: 0.0984 - val_mean_squared_error: 0.0185
Epoch 3/100
45/46 [============================>.] - ETA: 0s - loss: 0.1030 - mean_squared_error: 0.0197
Epoch 3: val_loss did not improve from 0.09845
46/46 [================

Epoch 1/100
45/46 [============================>.] - ETA: 0s - loss: 0.1078 - mean_squared_error: 0.0235
Epoch 1: val_loss improved from inf to 0.10258, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch64_cnn3_filt32_size15_pool5_do0.1_tra5_head8_kdim256_fnn64/weights_3.hdf5
46/46 [==============================] - 16s 70ms/step - loss: 0.1076 - mean_squared_error: 0.0234 - val_loss: 0.1026 - val_mean_squared_error: 0.0194
Epoch 2/100
45/46 [============================>.] - ETA: 0s - loss: 0.1013 - mean_squared_error: 0.0193
Epoch 2: val_loss did not improve from 0.10258
46/46 [==============================] - 3s 55ms/step - loss: 0.1015 - mean_squared_error: 0.0194 - val_loss: 0.1026 - val_mean_squared_error: 0.0194
Epoch 3/100
45/46 [============================>.] - ETA: 0s - loss: 0.1016 - mean_squared_error: 0.0194
Epoch 3: val_loss did not improve from 0.10258
46/46 [================

OSError: [Errno 39] Directory not empty: 'randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/batch64_cnn3_filt32_size15_pool5_do0.1_tra5_head8_kdim256_fnn64' -> 'randomSearch/SRATE500_ECG-bandpass_z-norm_child_male_train/CNN+transformer_age%20(sigmoid)_loss(mae)-nodecay_4fold_500cases/mae2.15+-0.00_max0__batch64_cnn3_filt32_size15_pool5_do0.1_tra5_head8_kdim256_fnn64'

## BRNO team

In [5]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'
GENDER = 'male'

SMALL  = ''


hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'
FILENAME = f'dataset/{ADULT}_train_{GENDER}{SMALL}.npz'
with np.load(open(FILENAME, 'rb'), allow_pickle=True) as f:
    caseid = f['caseid']
    x_feature, x_signal, y = f['X_feature'], f['X_signal'], f['y']

    
x_signal = np.delete(x_signal, 5559, axis=0)
x_feature = np.delete(x_feature, 5559, axis=0)
y = np.delete(y, 5559, axis=0)
caseid = np.delete(caseid, 5559, axis=0)        
    
x_train = x_signal
print(caseid.shape, x_feature.shape, x_signal.shape, y.shape)

(13390,) (13390, 55) (13390, 5000, 12) (13390,)


In [6]:
# folder
nfold = 4  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 500
rootdir = f"randomSearch/{hyper_path}/CNN_4layers_{nfold}fold_test{ntest}"

In [10]:
from tensorflow.keras.optimizers import Adam
from keras.layers import MultiHeadAttention, GlobalMaxPool1D, Concatenate, LeakyReLU, AveragePooling1D
from tensorflow.keras.callbacks import LearningRateScheduler

seed_everything(SEED)

ntrial = ntest
train_errs, val_errs = [] ,[]
random_settings = []


odir_f = 'BRNO_same'
odir = rootdir + '/' + odir_f
if not os.path.exists(odir):
    os.mkdir(odir)


# ==================================================
# BUILD MODEL
# ==================================================
# input layer
inp_sig = Input(shape=(x_train.shape[1], x_train.shape[2]))
out = inp_sig

# first conv1d layer
out = Conv1D(filters=256, kernel_size=15, strides=2, padding='same', activation=None)(out)
out = BatchNormalization()(out)
out = LeakyReLU(alpha=0.01)(out)

# stacked residual blocks
for i in range(1, 6):
    res = out

    out = Conv1D(filters=256, kernel_size=9, strides=2, padding='same', activation=None)(out)
    out = BatchNormalization()(out)
    out = LeakyReLU(alpha=0.01)(out)
    out = Conv1D(filters=256, kernel_size=9, strides=1, padding='same', activation=None)(out)
    out = BatchNormalization()(out)
    out = LeakyReLU(alpha=0.01)(out)

    res = AveragePooling1D(pool_size=2, padding='same')(res)
    res = Conv1D(filters=256, kernel_size=1, strides=1, padding='same', activation=None)(res)

    out = res + out

# multi-head attention
out = Dropout(0.5)(out)
out = MultiHeadAttention(num_heads=8, key_dim=256)(out, out)
out = GlobalMaxPool1D()(out)

# concat with X_feature
inp_feat = Input(shape=(x_feature.shape[1]))
out = Concatenate(axis=1)([out, inp_feat])

# final dense layer
out = Dropout(0.5)(out)
out = Dense(256, activation=None)(out)
#out = BatchNormalization()(out)
#out = LeakyReLU(alpha=0.01)(out)

#out = Dropout(dropout_fc)(out)
#out = Dense(dense_node, activation=None)(out)
#out = LeakyReLU(alpha=0.01)(out)
out = Dense(1, activation='linear')(out)

# build model
model = Model(inputs=[inp_sig, inp_feat], outputs=[out])
model.save_weights(f'{odir}/initial_weights.hdf5')

# ==================================================
# K FOLD CROSS VALIDATION
# ==================================================
kfold = KFold(n_splits=nfold)
tprs, aucs, prs = [], [], []
test_rmse, test_mae = [], []
f1_scores, thvals = [], []
mean_fpr = np.linspace(0,1,100)
mean_recall = np.linspace(0,1,100)

switch = 0
caseids_train = np.unique(caseid)
for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
    c_cv_trains = caseids_train[c_cv_trains_mask]
    random.shuffle(c_cv_trains)

    ncv = int(len(c_cv_trains)*0.8)
    c_cv_train = c_cv_trains[:ncv]
    c_cv_val = c_cv_trains[ncv:]

    cv_train_mask = np.isin(caseid, c_cv_train)
    cv_val_mask = np.isin(caseid, c_cv_val)
    cv_test_mask = np.isin(caseid, caseids_train[c_cv_test_mask])

    X_train_signal = x_train[cv_train_mask]
    X_train_feature = x_feature[cv_train_mask]

    X_val_signal = x_train[cv_val_mask]
    X_val_feature = x_feature[cv_val_mask]

    X_test_signal = x_train[cv_test_mask]
    X_test_feature = x_feature[cv_test_mask]

    Y_train = y[cv_train_mask]
    Y_val = y[cv_val_mask]
    Y_test = y[cv_test_mask]


    # ==================================================
    # TRAIN MODEL
    # ==================================================

    def step_decay(epoch):
        start = 1e-3
        drop = 0.1
        epochs_drop = 10
        lr = start * (drop ** np.floor((epoch)/epochs_drop))
        return lr
    lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
    
    weightcache = f"{odir}/weights_{fold}.hdf5"
    model.compile(loss='mean_absolute_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_squared_error'])
    hist = model.fit([X_train_signal, X_train_feature], Y_train, validation_data=([X_val_signal, X_val_feature], Y_val), epochs=100, batch_size=256,
                     callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                     EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

    print('next step')
    model.load_weights(weightcache)
    y_pred = model.predict([X_test_signal, X_test_feature]).flatten()

    print('mae cal')
    # MAE 계산

    model_err = metrics.MeanAbsoluteError()
    model_err.update_state(Y_test, y_pred)
    mae_val = model_err.result().numpy()
    test_mae.append(mae_val)
    print('rmse cal')
    # RMSE 계산
    model_err = metrics.RootMeanSquaredError()
    model_err.update_state(Y_test, y_pred)
    rmse_val = model_err.result().numpy()
    test_rmse.append(rmse_val)


    print(f' ###{fold} fold : val mae {mae_val:.2f}, val rmse {rmse_val:.2f}###')
    tf.keras.backend.clear_session()
    model.load_weights(f'{odir}/initial_weights.hdf5')


# RMSE 계산
mean_rmse = np.mean(test_rmse)
std_rmse = np.std(test_rmse)

mean_mae = np.mean(test_mae)
std_mae = np.std(test_mae)

max_idx = test_mae.index(min(test_mae))


print(f'mae{mean_mae:.2f}+-{std_mae:.2f}_rmse{mean_rmse:.2f}+-{std_rmse:.2f}')
open(odir+"/model.json", "wt").write(model.to_json())

os.rename(odir, rootdir+f'/mae{mean_mae:.2f}+-{std_mae:.2f}_rmse{mean_rmse:.2f}+-{std_rmse:.2f}_max{max_idx}__{odir_f}')
tf.keras.backend.clear_session()

Epoch 1/100


2023-09-21 00:02:07.423584: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-21 00:02:07.884944: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-21 00:02:09.361620: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-21 00:02:09.374769: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1e17c2c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-21 00:02:09.374806: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-21 00:02:09.383599: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-21 00:02:09.448974: I tensor

32/32 [==============================] - ETA: 0s - loss: 22.0945 - mean_squared_error: 1465.9642
Epoch 1: val_loss improved from inf to 159.81340, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_4fold_test500/BRNO_same/weights_0.hdf5
32/32 [==============================] - 31s 517ms/step - loss: 22.0945 - mean_squared_error: 1465.9642 - val_loss: 159.8134 - val_mean_squared_error: 48110.7031
Epoch 2/100
32/32 [==============================] - ETA: 0s - loss: 11.9311 - mean_squared_error: 223.9284
Epoch 2: val_loss improved from 159.81340 to 24.37708, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_4fold_test500/BRNO_same/weights_0.hdf5
32/32 [==============================] - 14s 428ms/step - loss: 11.9311 - mean_squared_error: 223.9284 - val_loss: 24.3771 - val_mean_squared_error: 732.1544
Epoch 3/100
32/32 [==============================] - ETA: 0s - loss: 10.4996 - mean_squared_error: 178.0850
Epoch 3: v

Epoch 1/100
32/32 [==============================] - ETA: 0s - loss: 22.0408 - mean_squared_error: 1379.5649
Epoch 1: val_loss improved from inf to 37.47163, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_4fold_test500/BRNO_same/weights_1.hdf5
32/32 [==============================] - 24s 475ms/step - loss: 22.0408 - mean_squared_error: 1379.5649 - val_loss: 37.4716 - val_mean_squared_error: 1910.1138
Epoch 2/100
32/32 [==============================] - ETA: 0s - loss: 11.5303 - mean_squared_error: 210.6608
Epoch 2: val_loss improved from 37.47163 to 18.31047, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_4fold_test500/BRNO_same/weights_1.hdf5
32/32 [==============================] - 14s 446ms/step - loss: 11.5303 - mean_squared_error: 210.6608 - val_loss: 18.3105 - val_mean_squared_error: 447.1162
Epoch 3/100
32/32 [==============================] - ETA: 0s - loss: 10.6726 - mean_squared_error: 183.8148
Ep

Epoch 1/100
32/32 [==============================] - ETA: 0s - loss: 22.2764 - mean_squared_error: 1410.8519
Epoch 1: val_loss improved from inf to 88.39001, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_4fold_test500/BRNO_same/weights_2.hdf5
32/32 [==============================] - 25s 498ms/step - loss: 22.2764 - mean_squared_error: 1410.8519 - val_loss: 88.3900 - val_mean_squared_error: 10888.9648
Epoch 2/100
32/32 [==============================] - ETA: 0s - loss: 11.5786 - mean_squared_error: 209.5145
Epoch 2: val_loss improved from 88.39001 to 20.21701, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_4fold_test500/BRNO_same/weights_2.hdf5
32/32 [==============================] - 14s 437ms/step - loss: 11.5786 - mean_squared_error: 209.5145 - val_loss: 20.2170 - val_mean_squared_error: 522.8632
Epoch 3/100
32/32 [==============================] - ETA: 0s - loss: 10.6558 - mean_squared_error: 181.6925
E

Epoch 1/100
32/32 [==============================] - ETA: 0s - loss: 21.6108 - mean_squared_error: 1296.2366
Epoch 1: val_loss improved from inf to 376.34180, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_4fold_test500/BRNO_same/weights_3.hdf5
32/32 [==============================] - 24s 467ms/step - loss: 21.6108 - mean_squared_error: 1296.2366 - val_loss: 376.3418 - val_mean_squared_error: 154868.8750
Epoch 2/100
32/32 [==============================] - ETA: 0s - loss: 11.3763 - mean_squared_error: 207.7018
Epoch 2: val_loss improved from 376.34180 to 14.93443, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_4fold_test500/BRNO_same/weights_3.hdf5
32/32 [==============================] - 14s 437ms/step - loss: 11.3763 - mean_squared_error: 207.7018 - val_loss: 14.9344 - val_mean_squared_error: 392.3897
Epoch 3/100
32/32 [==============================] - ETA: 0s - loss: 10.9248 - mean_squared_error: 189.95

In [ ]:
def step_decay(epoch):
    start = 1e-3
    drop = 0.1
    epochs_drop = 20
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr